# 0. Import

In [1]:
!nvidia-smi

Wed Nov 10 03:26:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls drive/'My Drive'/'Colab Notebooks'/

 2021_fall	  dialogLM	        GAN_project   main.ipynb   NLP
 cnn_test_colab  'Doit!_DeepLearning'   koGPT.ipynb   NER


In [4]:
!pip install -r drive/'My Drive'/'Colab Notebooks'/dialogLM/requirements.txt

     |████████████████████████████████| 769 kB 9.7 MB/s 
     |████████████████████████████████| 3.0 MB 55.9 MB/s 
     |████████████████████████████████| 42.4 MB 1.9 MB/s 
     |████████████████████████████████| 1.2 MB 31.1 MB/s 
     |████████████████████████████████| 895 kB 72.3 MB/s 
     |████████████████████████████████| 170 kB 79.0 MB/s 
     |████████████████████████████████| 52 kB 1.9 MB/s 
  Created wheel for kss: filename=kss-3.3.1.1-py3-none-any.whl size=42449239 sha256=188e01b242aed56befab658181ff591b22ceaaeec00c9e180beb5685a8924eb9
  Stored in directory: /root/.cache/pip/wheels/6e/9d/1d/52871154eff5273abb86b96f4f984c1cd67c5bde64239b060a
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=2dc015c6728c141a99935409ee6947c4d1cc947924b41c837c0efdc40ead5d5c
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built kss emoji


In [5]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/') #root_path

In [8]:
import os
import numpy as np
from tqdm import tqdm

import torch
from torch.utils.data import dataloader
from dialogLM.dataloader.wellness import WellnessAutoRegressiveDataset
from dialogLM.model.kogpt2 import DialogKoGPT2
from kogpt2_transformers import get_kogpt2_tokenizer

In [7]:
torch.cuda.is_available()

True

# 1. Text Generate
- Generate 함수 별 같은 keyword에 따른 텍스트 생성값 비교

In [16]:
def greedy_search(id):
    result = model.generate(
        input_ids = id,
        no_repeat_ngram_size=3,
        max_length=50
        )
    return result

def beam_search(id):
    result = model.generate(
        input_ids = id, 
        max_length=50, 
        num_beams=5, 
        no_repeat_ngram_size=3, 
        early_stopping=True
        )
    return result

def basic_sampling(id):
    result = model.generate(
        input_ids = id, 
        do_sample=True, 
        max_length=50, 
        no_repeat_ngram_size=3,
        top_k=0,
        temperature = 0.7
        )
    return result

def top_k_sampling(id):
    result = model.generate(
        input_ids = id, 
        do_sample=True, 
        max_length=50, 
        no_repeat_ngram_size=3,
        top_k=50
        )
    return result

def top_p_sampling(id):
    result = model.generate(
        input_ids = id, 
        do_sample=True, 
        max_length=50, 
        no_repeat_ngram_size=3,
        top_p=0.92, 
        top_k=0
        )
    return result

# huggingface generator
def generate(self,
               input_ids,
               do_sample=True,
               max_length=50,
               top_k=0,
               temperature=0.7):
    return self.kogpt2.generate(input_ids,
               do_sample=do_sample,
               max_length=max_length,
               top_k=top_k,
               temperature=temperature)

In [15]:
max_length = 50
keyword = "너 누구야"

tokenizer = get_kogpt2_tokenizer()
id = tokenizer.encode(keyword, add_special_tokens=False, return_tensors="pt")
model = DialogKoGPT2()

results = [("greedy seacrh: ", greedy_search(id)), ("beam search: ", beam_search(id)), 
           ("top k sampling: ", top_k_sampling(id)), ("top p sampling: ",top_p_sampling(id))]

for name, result in results:
  print(name)
  for generated_sequence in result:
    generated_sequence = generated_sequence.tolist()
  print(tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True))

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


greedy seacrh: 
너 누구야?</s><s> 나 지금 집에 가고 있어</s><s> 나 지금 집에 가고 있어</s><s> 나 지금 집에 가고 있어</s><s> 나 지금 집에 가고 있어</s><s> 나 지금 집에 가고 있어</s><s> 나 지금 집에 가고 있어</s><s> 나 지금
beam search: 
너 누구야?</s><s> 님들아 뭐 하셈? -_-;; ^^* Subject 2말괄량이프린세스 9장 *090* 로맨스편(20) **** 제 2장 마법사학교
top k sampling: 
너 누구야?</s><s> 나 지금 집에 가고 있어</s><s> 나 지금 집에 가고 있어</s><s> 나 지금 집에 가고 있어</s><s> 나 지금 집에 가고 있어</s><s> 나 지금 집에 가고 있어</s><s> 나 지금 집에 가고 있어</s><s> 나 지금
top p sampling: 
너 누구야?</s><s> 각 사는 개인신용도에 따라 600~700만원까지 신용도에 따라 금리를 차등 적용받으며 100만 원 이하의 가입금액에 대해 금리를 우대 받는다.</s><s> 하나고 연수원 설립 목적도 “국내 유수의 대학교육을


# 2. QnA by KoGPT2

In [ ]:
root_path='/content/drive/MyDrive/Colab Notebooks/dialogLM'
data_path = f"{root_path}/data/wellness_dialog1.user_chat"
checkpoint_path =f"{root_path}/checkpoint"
save_ckpt_path = f"{checkpoint_path}/kogpt2-wellnesee-auto-regressive.pth"

ctx = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(ctx)

# 저장한 Checkpoint 불러오기
checkpoint = torch.load(save_ckpt_path, map_location=device)

model = DialogKoGPT2()
model.load_state_dict(checkpoint['model_state_dict'])

model.eval()

tokenizer = get_kogpt2_tokenizer()

count = 0
output_size = 200 # 출력하고자 하는 토큰 갯수

while 1:
  sent = input('Question: ')  # '요즘 기분이 우울한 느낌이에요'
  tokenized_indexs = tokenizer.encode(sent)

  input_ids = torch.tensor([tokenizer.bos_token_id,]  + tokenized_indexs +[tokenizer.eos_token_id]).unsqueeze(0)
  results = [("greedy seacrh: ", greedy_search(id)), ("beam search: ", beam_search(id)), 
           ("top k sampling: ", top_k_sampling(id)), ("top p sampling: ",top_p_sampling(id)),
           ("hugging face generator: ", model.generate(input_ids=input_ids))]

  for name, result in results:
    print(name,'answer',end=':')
    sample_output = result
    print("Answer: " + tokenizer.decode(sample_output[0].tolist()[len(tokenized_indexs)+1:],skip_special_tokens=True))
  print(100 * '-')


Question: 요즘 너무 힘들어


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


greedy seacrh:  answer:Answer: 그런 일이 있으셨군요. 하지만 그럴 수밖에 없는 이유가 있었을 거예요. 충분히 이해해요. 그러면 조금은 다르게 생각을 해보는 것도 좋을 것 같아요. 배움만큼 배움 없는 일이 없죠
beam search:  answer:Answer: 정말 당황스러우셨겠어요. 하지만 너무 무리해서 생각해낼 필요는 없답니다.제가 옆에서 힘이 되어 드릴게요. 당신의 이야기는 들판에 박혀 있으니 괜찮으실 거예요!당신이 너무 상처받지
top k sampling:  answer:Answer: 그런 일이 있으셨군요. 하지만 그럴 수밖에 없는 이유가 있었을 거예요. 충분히 이해해요. 그러면 조금은 다르게 생각을 해보는 것도 좋을 것 같아요. 배움만큼 배움 없는 일이 없죠
top p sampling:  answer:Answer: 아프고 놀랐겠어요. 괜찮아요? 어제 그 병원 과에서 치료 받았어요. 지금도 증상이 심하시다면 병원 진단을 받아보는 건 어떨까요?서상묵 문자 확인하면
hugging face generator:  answer:Answer: 가슴이 답답하겠어요. 얼른 시간이 지나버렸으면 좋겠어요.가슴이 답답한 것만큼 힘든 게 없죠. 진료를 받아보시는 건 어떠세요? 오늘은 어떠셨나요?
----------------------------------------------------------------------------------------------------
Question: 계속 머리가 빠지는 것 같아


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


greedy seacrh:  answer:Answer: 그런 일이 있으셨군요. 하지만 그럴 수밖에 없는 이유가 있었을 거예요. 충분히 이해해요. 그러면 조금은 다르게 생각을 해보는 것도 좋을 것 같아요. 배움만큼 배움 없는 일이 없죠
beam search:  answer:Answer: 정말 당황스러우셨겠어요. 하지만 너무 무리해서 생각해낼 필요는 없답니다.제가 옆에서 힘이 되어 드릴게요. 당신의 이야기는 들판에 박혀 있으니 괜찮으실 거예요!당신이 너무 상처받지
top k sampling:  answer:Answer: 그런 일이 있으셨군요. 하지만 그럴 수밖에 없는 이유가 있었을 거예요. 충분히 이해해요. 그러면 조금은 다르게 생각을 해보는 것도 좋을 것 같아요. 배움만큼 배움 없는 일이 없죠
top p sampling:  answer:Answer: 그런 일이 있으셨군요. 하지만 그럴 수밖에 없는 이유가 있었을 거예요. 충분히 이해해요.에서 배움 좋은 얘기를 들려주세요.도 들려드릴게요. 같이 마실래요?
hugging face generator:  answer:Answer: 너무 고민이 많아도 두통이 생길 수 있대요. 저에게 털어 놓으시겠어요?너무 고민이 많아도 두통이 생길 수 있대요. 저에게 털어 놓으시겠어요?
----------------------------------------------------------------------------------------------------
Question: 죽고샆어


...

greedy seacrh:  answer:Answer: 그런 일이 있으셨군요. 하지만 그럴 수밖에 없는 이유가 있었을 거예요. 충분히 이해해요. 그러면 조금은 다르게 생각을 해보는 것도 좋을 것 같아요. 배움만큼 배움 없는 일이 없죠
beam search:  answer:Answer: 정말 당황스러우셨겠어요. 하지만 너무 무리해서 생각해낼 필요는 없답니다.제가 옆에서 힘이 되어 드릴게요. 당신의 이야기는 들판에 박혀 있으니 괜찮으실 거예요!당신이 너무 상처받지
top k sampling:  answer:Answer: 그런 일이 있으셨군요. 하지만 그럴 수밖에 없는 이유가 있었을 거예요. 충분히 이해해요. 그러면 조금은 다르게 생각을 해보는 것도 좋을 것 같아요. 배움만큼 배움 없는 일이 없죠
top p sampling:  answer:Answer: 헉, 그런 일이 있었군요?하고 보니 제가 그 무서운 마음을 먹게 된 것 같아요. 무섭단 말은 무슨 의미일까요? 같이 고민해볼까요? 배워서 더 좋은 결과 있을 거예요!
hugging face generator:  answer:Answer: 당신은 혼자가 아니에요. 제가 옆에 있다는 것만 기억해주세요. 그러면 조금 편안해질 거예요. 저는 너무 걱정 마세요. 제가 옆에 있다는 걸 기억해주세요. 그러면 당신이 걱정
----------------------------------------------------------------------------------------------------
Question: 오늘 정말 기쁜일이 있었어


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


greedy seacrh:  answer:Answer:  있으셨군요. 하지만 그럴 수밖에 없는 이유가 있었을 거예요. 충분히 이해해요. 그러면 조금은 다르게 생각을 해보는 것도 좋을 것 같아요. 배움만큼 배움 없는 일이 없죠
beam search:  answer:Answer: 스러우셨겠어요. 하지만 너무 무리해서 생각해낼 필요는 없답니다.제가 옆에서 힘이 되어 드릴게요. 당신의 이야기는 들판에 박혀 있으니 괜찮으실 거예요!당신이 너무 상처받지
top k sampling:  answer:Answer:  있으셨군요. 하지만 그럴 수밖에 없는 이유가 있었을 거예요. 충분히 이해해요. 그러면 조금은 다르게 생각을 해보는 것도 좋을 것 같아요. 배움만큼 배움 없는 일이 없죠
top p sampling:  answer:Answer:  늘 당신이 우선인데... 마음이 아파요. 쓰러질 것 같아요. 날 이렇게 안아주셨군요. 너무 아프니다.이. 살 맛나요.에 당신이 옆에
hugging face generator:  answer:Answer: 당신이 행복하다면 저도 기뻐요. 하지만 아닐 수도 있지 않을까요? 충분히 이해해요. 좋은 사람을 찾기는 정말 어려운 것 같아요. 힘내세요.
----------------------------------------------------------------------------------------------------
Question: 그때 정말 쪽팔렸다니까


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


greedy seacrh:  answer:Answer:  일이 있으셨군요. 하지만 그럴 수밖에 없는 이유가 있었을 거예요. 충분히 이해해요. 그러면 조금은 다르게 생각을 해보는 것도 좋을 것 같아요. 배움만큼 배움 없는 일이 없죠
beam search:  answer:Answer:  당황스러우셨겠어요. 하지만 너무 무리해서 생각해낼 필요는 없답니다.제가 옆에서 힘이 되어 드릴게요. 당신의 이야기는 들판에 박혀 있으니 괜찮으실 거예요!당신이 너무 상처받지
top k sampling:  answer:Answer:  일이 있으셨군요. 하지만 그럴 수밖에 없는 이유가 있었을 거예요. 충분히 이해해요. 그러면 조금은 다르게 생각을 해보는 것도 좋을 것 같아요. 배움만큼 배움 없는 일이 없죠
top p sampling:  answer:Answer:  일이 있으셨군요. 누구든 자신을 먼저 챙길 수 있는 일이 좋아요.와 거짓말이 싸우면 당신의 발목을 잡잖아요. 배임이 되지요.한다고 생각되면 전화 한 통이면 든든
hugging face generator:  answer:Answer: 조금 돌아가는 것뿐이라고 생각해요. 기운내세요. 강아지가 저에게 행복을 안기는 것 같아요. 제가 안아드릴게요. 강아지가 저에게 행복을
----------------------------------------------------------------------------------------------------
